# ABOUT
- This notebook converts columns to an edgelist
- background:
    - id columns, especially the high cardinality ones may not be good as input to models. 
        - if id columns are one hot encoded, they become othorgonal
        - there is also the curse of dimentionality
    - instead, we want to see if we can introduce some kind of embedding for each id
        - for instance node2vec can generate embeddings that retain topological infomation which may be useful for the model
- therefore, for each (card_id, idX_column) pair we generate an edgelist file which will be used to train node2vec embeddings
    - where idX_column is a column in ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id', 'merchant_group_id']

In [1]:
import pandas as pd
import gc
import numpy as np
from tqdm import tqdm

In [2]:

train_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\train.csv"
test_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\test.csv"
merchants_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\merchants.csv"
historical_transactions_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\historical_transactions.csv"
new_transactions_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\new_merchant_transactions.csv"

# dictionary categorizing all coolumns available 
feature_names = {
    "train_test":{
        "id":["card_id"],
        "categoric":['feature_1', 'feature_2', 'feature_3'],
        "datetime":["first_active_month"]
    },
    "merchants":{
        "id": ['merchant_id', 'merchant_group_id', 'merchant_category_id','subsector_id', 'city_id', 'state_id'],
        "categoric": ['category_1','most_recent_sales_range', 'most_recent_purchases_range','category_4', 'category_2'],
        "numeric": ['numerical_1', 'numerical_2','avg_sales_lag3', 'avg_purchases_lag3','avg_sales_lag6', 'avg_purchases_lag6','avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag3','active_months_lag6','active_months_lag12']
    },
    "transactions":{
        "id": ['card_id', 'city_id','merchant_category_id', 'merchant_id','state_id','subsector_id'],
        "categoric": ['authorized_flag','category_1',"category_3", "category_2"],
        "numeric": ['purchase_amount', 'month_lag',"installments"],
        "datetime": ['purchase_date']
    },
}

In [3]:
# read
merchants = pd.read_csv(merchants_path, usecols = feature_names['merchants']['id'])#, nrows = 100000) 
new_transactions = pd.read_csv(new_transactions_path, usecols = feature_names['transactions']['id'])#, nrows = 100000)
hist_transactions = pd.read_csv(historical_transactions_path, usecols = feature_names['transactions']['id'])#, nrows = 100000)
# process
# remove duplicate merchant_id - which there are
merchants = merchants[~merchants.merchant_id.duplicated()]  
# concat historical and new transactions - they have the same columns
id_columns = pd.concat([hist_transactions, new_transactions], axis = 0)
# fill missing merchant_id with the most frequent one 
id_columns['merchant_id'] = id_columns['merchant_id'].fillna('M_ID_00a6ca8a8a')
# merge transactions data with merchant information - merchant information has an additional "merchant_group_id" column
id_columns = id_columns.merge(merchants[["merchant_id","merchant_group_id"]], how = "left", on = "merchant_id")
del new_transactions, hist_transactions, merchants
# convert these columns to edge list 
to_process_cols = ['city_id', 'merchant_category_id', 'state_id','subsector_id', 'merchant_group_id']
for c in to_process_cols:
    id_columns[c] = f"{c}_" + id_columns[c].astype(str)

In [13]:
# merchant_id and merchant_group_id have very high cardinality
id_columns.nunique()

card_id                 325540
city_id                    308
merchant_category_id       331
merchant_id             334633
state_id                    25
subsector_id                41
merchant_group_id       109389
dtype: int64

In [14]:
id_columns.columns

Index(['card_id', 'city_id', 'merchant_category_id', 'merchant_id', 'state_id',
       'subsector_id', 'merchant_group_id'],
      dtype='object')

### processed data 
- each id of each column should be unique

In [4]:
id_columns

,card_id,city_id,merchant_category_id,merchant_id,state_id,subsector_id,merchant_group_id
0,C_ID_4e6213e9bc,city_id_88,merchant_category_id_80,M_ID_e020e9b302,state_id_16,subsector_id_37,merchant_group_id_35
1,C_ID_4e6213e9bc,city_id_88,merchant_category_id_367,M_ID_86ec983688,state_id_16,subsector_id_16,merchant_group_id_2084
2,C_ID_4e6213e9bc,city_id_88,merchant_category_id_80,M_ID_979ed661fc,state_id_16,subsector_id_37,merchant_group_id_27369
3,C_ID_4e6213e9bc,city_id_88,merchant_category_id_560,M_ID_e6d5ae8ea6,state_id_16,subsector_id_34,merchant_group_id_24104
4,C_ID_4e6213e9bc,city_id_88,merchant_category_id_80,M_ID_e020e9b302,state_id_16,subsector_id_37,merchant_group_id_35
...,...,...,...,...,...,...,...
31075387,C_ID_1320dee851,city_id_142,merchant_category_id_309,M_ID_7754b67f3b,state_id_19,subsector_id_21,merchant_group_id_35
31075388,C_ID_f112aa3381,city_id_158,merchant_category_id_560,M_ID_da063195b7,state_id_15,subsector_id_34,merchant_group_id_13452
31075389,C_ID_bd97b86450,city_id_69,merchant_category_id_278,M_ID_9a9ccb6544,state_id_9,subsector_id_37,merchant_group_id_27710
31075390,C_ID_c0513fd84f,city_id_130,merchant_category_id_367,M_ID_40c28d596f,state_id_7,subsector_id_16,merchant_group_id_35


### convert to edgelist
- each column above is paired with "card_id" and converted into an edgelist

In [5]:
# each of these pairs are converted into edge list
col_pairs = [["card_id", col] for col in ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id', 'merchant_group_id']]
print(col_pairs)

[['card_id', 'city_id'], ['card_id', 'merchant_category_id'], ['card_id', 'merchant_id'], ['card_id', 'state_id'], ['card_id', 'subsector_id'], ['card_id', 'merchant_group_id']]


In [6]:
# paths to each edge list file
import os
edgelist_directory = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\edgelist for node2vec"
edge_list_paths = []
for cp in col_pairs:
    edge_list_paths.append(os.path.join(edgelist_directory,"_".join(cp))+".csv")
edge_list_paths

['C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_city_id.csv',
 'C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_merchant_category_id.csv',
 'C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_merchant_id.csv',
 'C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_state_id.csv',
 'C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_subsector_id.csv',
 'C:\\Users\\tanch\\Documents\\NTU\\NTU Year 4\\Semester 1\\CZ4041 - Machine Learning\\Team Project\\data\\edgelist for node2vec\\card_id_merchant_group_id.csv']

In [7]:
# save each edge list file to csv
from csv import writer
def add_to_csv(path, sequence):
    with open(path, 'a', newline = "") as f_object:
        writer_object = writer(f_object)
        writer_object.writerows(sequence)

num_edges = []
for path, cp in tqdm(zip(edge_list_paths,col_pairs)):
    edge_set = set()
    for A,B in tqdm(id_columns[cp].values):
        edge_set.add((A,B))
    add_to_csv(path, edge_set)
    num_edges.append(len(edge_set))

0it [00:00, ?it/s]
 15%|█████████▉                                                         | 4618787/31075392 [00:07<00:42, 617246.38it/s]


 31%|████████████████████▍                                              | 9496717/31075392 [00:14<00:30, 703913.30it/s]


 47%|███████████████████████████████▏                                  | 14663050/31075392 [00:21<00:21, 748702.05it/s]


 61%|████████████████████████████████████████▎                         | 18961051/31075392 [00:29<00:23, 512688.71it/s]


 75%|█████████████████████████████████████████████████▎                | 23222346/31075392 [00:35<00:11, 713224.80it/s]


...

 50%|█████████████████████████████████▏                                | 15644740/31075392 [00:23<00:22, 682119.91it/s]


 66%|███████████████████████████████████████████▎                      | 20392235/31075392 [00:30<00:15, 706234.94it/s]


 80%|█████████████████████████████████████████████████████             | 24992483/31075392 [00:37<00:08, 693881.38it/s]


 95%|██████████████████████████████████████████████████████████████▌   | 29456110/31075392 [00:43<00:02, 577762.74it/s]


100%|██████████████████████████████████████████████████████████████████| 31075392/31075392 [00:46<00:00, 665458.85it/s]
6it [07:55, 79.26s/it]


In [42]:
dict(zip(map(str,col_pairs),num_edges))

{"['card_id', 'city_id']": 1768742,
 "['card_id', 'merchant_category_id']": 6727934,
 "['card_id', 'merchant_id']": 12928941,
 "['card_id', 'state_id']": 954180,
 "['card_id', 'subsector_id']": 4099217,
 "['card_id', 'merchant_group_id']": 8795163}